In [1]:
import sys
import re
print(sys.getrecursionlimit())
sys.setrecursionlimit(1000000)

3000


In [2]:
class FSM:

    def __init__(self, input):
        # Shrani input in inicializiraj simbol, ki ga KA trenutno bere, skupaj z njegovim indeksom.
        # Nastavi output na prazni string.
        self.tape = input
        self.symbol = input[0]
        self.index = 0

    def next(self):
        # Preberi naslednji simbol
        self.index += 1
        try:
            self.symbol = self.tape[self.index]
        except IndexError:
            self.symbol = "_"

    def back(self):
        # Preberi prejšnji simbol
        self.index -= 1
        if self.index >= 0:
            self.symbol = self.tape[self.index]
        else: 
            self.symbol = "_"

    def start(self):
        # Začetno stanje.
        # Če prebere "A", izpiše "#" in se premakne dalje v stanje A1
        if self.symbol == "A":
            self.tape = self.tape[: self.index] + "#" + self.tape[self.index + 1:]
            self.next()
            return self.A1()
        # Če pride do konca vhodnega niza, gre v končno stanje
        elif self.symbol == "_":
            return self.end()
        # Če prebere kar koli drugega, izpiše "O" in začne iskati "AAUAUG" oz. "AACAUG" od začetka
        else:
            self.tape = self.tape[: self.index] + "O" + self.tape[self.index + 1:]
            self.next()
            return self.start()

    def A1(self):
        # Če prebere "A", črko ohrani in se premakne za eno v desno in v stanje A2
        if self.symbol == "A":
            self.next()
            return self.A2()
        # Sicer izbriši začetek trenutno opazovanega podniza in začni znova
        else:
            self.back()
            return self.delete()
        
    def A2(self):
        # Če prebere znak U ali C, ju ohrani in nadaljuj v stanje UC
        if self.symbol == "U" or self.symbol == "C":
            self.next()
            return self.UC()
        # Sicer izbriši začetek trenutno opazovanega podniza in začni znova
        else:
            self.back()
            return self.delete()
        
    def UC(self):
        if self.symbol == "A":
            self.next()
            return self.A3()
        else:
            self.back()
            return self.delete()
        
    def A3(self):
        if self.symbol == "U":
            self.next()
            return self.U1()
        else:
            self.back()
            return self.delete()
        
    def U1(self):
        # Stanje U1
        # Če je naslednji znak "G", ga ohrani in začni iskati substring od začetka
        if self.symbol == "G":
            self.next()
            return self.start()
        
        # Sicer izbriši začetek trenutno opazovanega podniza in začni znova
        else:
            self.back()
            return self.delete()
        
        
    def delete(self):
        # Ohrani simbole
        if self.symbol == "A" or self.symbol == "G" or self.symbol == "C" or self.symbol == "U":
            self.back()
            return self.delete()
        
        # Zbriši začetek in začni znova pri naslednjem simbolu
        elif self.symbol == "#":
            self.tape = self.tape[: self.index] + "O" + self.tape[self.index + 1:]
            self.next()
            return self.start()
        
    def end(self):
        return self.tape

In [13]:
# Program, ki prebere genom in poišče vse indekse "AUGUGG".
# Vrne seznam vseh indeksov, kjer se "AUGUGG" začne.
# Genom začnemo indeksirati z 0.

def find_indices(file):
    f = open(file, "r")
    genome = f.read()
    fsm = FSM(genome)
    output = fsm.start()

    print(genome[:-2])
    print(output)
    
    U = [m.start() for m in re.finditer('#AUAUG', output)]
    C = [m.start() for m in re.finditer('#ACAUG', output)]
    vse = [m.start() for m in re.finditer('#', output)]
 
    print("Začetni indeksi pojavitev 'AAUAUG': ", U)
    print("Začetni indeksi pojavitev 'AACAUG': ", C)
    print("Začetni indekri vseh pojavitev asparagina in metionina: ", vse)


file = "generated_genome.txt"
find_indices(file)

...
Začetni indeksi pojavitev 'AAUAUG':  [5802, 12686, 13491, 16546, 22791, 29068, 32418, 35313, 38512, 48406, 49426, 49480, 56505, 64416, 66670, 70362, 71221, 72727, 79836, 90810, 96221, 96797]
Začetni indeksi pojavitev 'AACAUG':  [87, 741, 2601, 20695, 27087, 33616, 38418, 39254, 40316, 47611, 50297, 52819, 59779, 60732, 69173, 71765, 72389, 73763, 77177, 77403, 77613, 83899, 84438, 84946, 85516, 95465, 98347, 99256]
Začetni indekri vseh pojavitev asparagina in metionina:  [87, 741, 2601, 5802, 12686, 13491, 16546, 20695, 22791, 27087, 29068, 32418, 33616, 35313, 38418, 38512, 39254, 40316, 47611, 48406, 49426, 49480, 50297, 52819, 56505, 59779, 60732, 64416, 66670, 69173, 70362, 71221, 71765, 72389, 72727, 73763, 77177, 77403, 77613, 79836, 83899, 84438, 84946, 85516, 90810, 95465, 96221, 96797, 98347, 99256]
